In [1]:

include("build_Yle_S.jl")
include("compute_Z_self.jl")
using SparseArrays, SuperLU, IterativeSolvers, FFTW, FourierTools
using LinearMaps, MAT, LinearAlgebra, Krylov, KrylovMethods



function FFT_solver_QS_S_type(freq, escalings, incidence_selection, FFTCP, FFTCLp, diagonals, ports, lumped_elements, expansions, GMRES_settings, Zs_info, QS_Rcc_FW)
    println(Base.Threads.nthreads())
    FFTW.set_num_threads(3)
    BLAS.set_num_threads(3)
    freq = freq .* escalings["freq"]
    # GMRES settings ----------------------------
    Inner_Iter = GMRES_settings.Inner_Iter
    Outer_Iter = GMRES_settings.Outer_Iter
    # -------------------------------------------
    m = size(incidence_selection["A"], 1)
    n = size(incidence_selection["A"], 2)
    ns = size(incidence_selection["Gamma"], 2)
    w = 2 .* pi .* freq
    nfreq = length(w)
    is = zeros(Float64, n, 1)
    S = zeros(ComplexF64, size(ports.port_nodes, 1), size(ports.port_nodes, 1), length(freq))
    Vrest = zeros(ComplexF64, m + n + ns, size(ports.port_nodes, 1))
    invP = sparse(1:ns, 1:ns, 1 ./ diagonals["P"],ns,ns)
    R_chiusura = 50.0
    
    for k = 1:nfreq
        println("Freq n=$k - Freq Tot=$nfreq")
        # Questa parte la vedremo in futuro (ignoratela per ora tanto QS_Rcc_FW è impostato ad 1)-------------------------------------------------------
        if QS_Rcc_FW == 2
            FFTCLp_rebuilted = compute_Circulant_Lp_Rcc(FFTCLp, escalings, freq[k] ./ escalings["freq"])
            FFTCP_rebuilted = compute_Circulant_P_sup_Rcc(FFTCP, escalings, freq[k] ./ escalings["freq"])
        elseif QS_Rcc_FW == 3
            FFTCLp_rebuilted = compute_Circulant_Lp_FW(FFTCLp, escalings, freq[k] ./ escalings["freq"])
            FFTCP_rebuilted = compute_Circulant_P_sup_FW(FFTCP, escalings, freq[k] ./ escalings["freq"])
        end
        # ---------------------------------------------------------------------------

        Yle = build_Yle_S(lumped_elements, [], ports, escalings, n, w[k] / escalings["freq"], R_chiusura)
        Z_self = compute_Z_self(diagonals["R"], diagonals["Cd"], w[k])
        Zs = escalings["R"] * (Zs_info["Zs"] * sqrt(w[k] / escalings["freq"]))
        ind_to_put_zero_Z_self = findall((real.(Zs[Zs_info["surface_edges"]]) .- real.(Z_self[Zs_info["surface_edges"]])) .> 0)
        ind_to_put_zero_Zs = findall((real.(Zs[Zs_info["surface_edges"]]) .- real.(Z_self[Zs_info["surface_edges"]])) .< 0)
        Z_self[Zs_info["surface_edges"][ind_to_put_zero_Z_self]] .= 0 .+ 1im * imag.(Z_self[Zs_info["surface_edges"][ind_to_put_zero_Z_self]])
        Zs[Zs_info["surface_edges"][ind_to_put_zero_Zs]] .= 0 .+ 1im * imag.(Zs[Zs_info["surface_edges"][ind_to_put_zero_Zs]])
        DZ = Z_self .+ real.(Zs)
        DZ .= DZ .+ 1im * w[k] * diagonals["Lp"]
        invZ = sparse(1:m, 1:m, 1 ./ DZ[:],m,m)
        
        # --------------------- preconditioner ------------------------
        SS = Yle+(prod_real_complex(transpose(incidence_selection["A"]) , prod_complex_real(invZ , incidence_selection["A"])) + 1im * w[k] * incidence_selection["Gamma"] * invP * transpose(incidence_selection["Gamma"]))
        F = LinearAlgebra.lu(SS)
        
        # --------------------------------------------------------------
        for c1 = 1:size(ports.port_nodes, 1)
            n1 = convert(Int64,ports.port_nodes[c1, 1])
            n2 = convert(Int64,ports.port_nodes[c1, 2])
            is[n1] = 1 * escalings["Is"]
            is[n2] = -1 * escalings["Is"]
            tn = precond_3_3_Kt(F, invZ, invP, incidence_selection["A"], incidence_selection["Gamma"], m, ns, is)
    
            products_law = x ->   ComputeMatrixVector(x, w[k], incidence_selection, FFTCP, FFTCLp, DZ, Yle, expansions, invZ, invP, F)
            prodts = LinearMap{ComplexF64}(products_law, n + m + ns, n + m + ns)
            #x0::Vector{ComplexF64}=Vrest[:, c1];
        
            if QS_Rcc_FW == 1
                #V, flag, relres, iter, resvec = gmres(ComputeMatrixVector, tn, Inner_Iter, GMRES_settings.tol[k], Outer_Iter, [], [], Vrest[:, c1], w[k], incidence_selection, FFTCP, FFTCLp, DZ, Yle, expansions, invZ, invP, L1, U1, P1, Q1)
                
                V, info = IterativeSolvers.gmres!(Vrest[:, c1], prodts, tn; reltol=GMRES_settings.tol[k], restart=Inner_Iter, maxiter=Inner_Iter, initially_zero=false, log=true, verbose=false)
                #V, info = IterativeSolvers.gmres(prodts, tn; reltol=GMRES_settings.tol[k], restart=Inner_Iter, maxiter=Inner_Iter, initially_zero=false, log=true, verbose=true)
                println(info)
                
                # (V, stats) = Krylov.gmres(prodts,tn,x0;restart=false, memory=Outer_Iter, reorthogonalization=false,rtol=GMRES_settings.tol[k], itmax=Inner_Iter,verbose=1, history=false)
            
                # if (stats.solved==true)        
                #     println("convergence reached, number of iterations: "*string(stats.niter))
                # else
                #     println("convergence not reached, number of iterations: "*string(stats.niter))
                # end

                #V,flag,err,iter,resvec = KrylovMethods.gmres(prodts,tn,Inner_Iter,tol=GMRES_settings.tol[k],maxIter=1,x=x0,out=1)

                # if (flag==0)        
                #     println("convergence reached, number of iterations: "*string(length(resvec)))
                # elseif(flag==-1)
                #     println("convergence not reached, number of iterations: "*string(length(resvec)))
                # else
                #     println("RHS all zeros, number of iterations: "*string(length(resvec)))
                # end
            else
                #V, flag, relres, iter, resvec = gmres(ComputeMatrixVector, tn, Inner_Iter, GMRES_settings.tol[k], Outer_Iter, [], [], Vrest[:, c1], w[k], incidence_selection, FFTCP_rebuilted, FFTCLp_rebuilted, DZ, Yle, expansions, invZ, invP, L1, U1, P1, Q1)
            end
            
            Vrest[:, c1] = V
            is[n1] = 0
            is[n2] = 0
            for c2 = c1:size(ports.port_nodes, 1)
                n3 = convert(Int64, ports.port_nodes[c2, 1])
                n4 = convert(Int64, ports.port_nodes[c2, 2])
                if c1 == c2
                    S[c1, c2, k] = (2 * (V[m+ns+n3] - V[m+ns+n4]) - R_chiusura) / R_chiusura
                else
                    S[c1, c2, k] = (2 * (V[m+ns+n3] - V[m+ns+n4])) / R_chiusura
                end
                S[c2, c1, k] = S[c1, c2, k]
            end
        end
    end
    out = Dict()
    out["S"] = S
    out["Z"] = s2z(S, R_chiusura)
    out["Y"] = s2y(S, R_chiusura)
    out["f"] = freq ./ escalings["freq"]
    return out
end

function ComputeMatrixVector(x_in, w, incidence_selection, FFTCP, FFTCLp, DZ, Yle, expansions, invZ, invP, lu)
    x::Vector{ComplexF64}=x_in[:,1];
    m = size(incidence_selection["A"], 1)
    ns = size(incidence_selection["Gamma"], 2)
    I = x[1:m]
    Q = x[m+1:m+ns]
    Phi = x[m+ns+1:end]
    # Lp * I ---------------------------------------------------------------
    mx = incidence_selection["mx"]
    my = incidence_selection["my"]
    mz = incidence_selection["mz"]
    Y1 = zeros(ComplexF64, m)
    ind_aux_Lp = Dict{Int,Vector{Int}}()
    ind_aux_Lp[1] = 1:mx
    ind_aux_Lp[2] = mx+1:mx+my
    ind_aux_Lp[3] = mx+my+1:mx+my+mz
    for cont = 1:3
            Nx = size(FFTCLp[cont, 1], 1) ÷ 2
            Ny = size(FFTCLp[cont, 1], 2) ÷ 2
            Nz = size(FFTCLp[cont, 1], 3) ÷ 2
            Ired = I[ind_aux_Lp[cont]]
            I_exp = prod_real_complex(expansions["mat_map_Lp"][cont, 1] , Ired)
            CircKT = reshape(I_exp, Nx, Ny, Nz)
            padded_CircKt = zeros(ComplexF64, 2*Nx,2*Ny,2*Nz)
            @views padded_CircKt[1:size(CircKT,1), 1:size(CircKT,2), 1:size(CircKT,3)] = CircKT
            Chi = ifft!(FFTCLp[cont, 1] .* fft!(padded_CircKt))
            Y1[ind_aux_Lp[cont]] = Y1[ind_aux_Lp[cont]] + prod_real_complex(transpose(expansions["mat_map_Lp"][cont, 1]) , reshape(Chi[1:Nx, 1:Ny, 1:Nz], Nx * Ny * Nz, 1))
    end
    Y1 = 1im * w * Y1 + DZ .* I + prod_real_complex(incidence_selection["A"] , Phi)
    # ---------------- P * Q ---------------------------------------------
    Y2 = zeros(ComplexF64,ns)
    for cont1 = 1:3
        for cont2 = cont1:3
            Nx = size(FFTCP[cont1, cont2], 1) ÷ 2
            Ny = size(FFTCP[cont1, cont2], 2) ÷ 2
            Nz = size(FFTCP[cont1, cont2], 3) ÷ 2
            Q_exp = prod_real_complex(expansions["exp_P"][cont1, cont2] , Q)
            CircKT = reshape(Q_exp, Nx, Ny, Nz)
            padded_CircKt = zeros(ComplexF64, 2*Nx,2*Ny,2*Nz)
            @views padded_CircKt[1:size(CircKT,1), 1:size(CircKT,2), 1:size(CircKT,3)] = CircKT
            Chi = ifft!(FFTCP[cont1, cont2] .* fft!(padded_CircKt))
            Y2 = Y2 + prod_real_complex(transpose(expansions["exp_P"][cont2, cont1]) , (reshape(Chi[1:Nx, 1:Ny, 1:Nz], Nx * Ny * Nz, 1)))
            if cont1 != cont2
                Q_exp = prod_real_complex(expansions["exp_P"][cont2, cont1] , Q)
                CircKT = reshape(Q_exp, Nx, Ny, Nz)
                padded_CircKt = zeros(ComplexF64, 2*Nx,2*Ny,2*Nz)
                @views padded_CircKt[1:size(CircKT,1), 1:size(CircKT,2), 1:size(CircKT,3)] = CircKT
                Chi = ifft!(FFTCP[cont1, cont2] .* fft!(padded_CircKt))
                Y2 = Y2 + prod_real_complex(transpose(expansions["exp_P"][cont1, cont2]) , (reshape(Chi[1:Nx, 1:Ny, 1:Nz], Nx * Ny * Nz, 1)))
            end
        end
    end
    Y2 = Y2 - prod_real_complex(transpose(incidence_selection["Gamma"]) , Phi)
    Y3 = -1.0*(prod_real_complex(transpose(incidence_selection["A"]) , I)) + prod_real_complex(Yle , Phi) + 1im * w * (prod_real_complex(incidence_selection["Gamma"] , Q))
    MatrixVector = precond_3_3_vector(lu, invZ, invP, incidence_selection["A"], incidence_selection["Gamma"], w, Y1, Y2, Y3)
    return MatrixVector
end


function precond_3_3_Kt(F, invZ, invP, A,Gamma, n1,n2, X3)

    n3 = length(X3)

    i1 = range(1, stop=n1)
    i2 = range(n1+1, stop=n1 + n2)
    i3 = range(n1 + n2 + 1, stop=n1 + n2 + n3)

    Y = zeros(ComplexF64, n1 + n2 + n3)

    M5 = F\X3

    Y[i1] .= Y[i1] .- 1.0*(prod_real_complex(invZ, prod_real_complex(A, M5)))
    Y[i2] .= Y[i2] .+ (prod_real_complex(invP, prod_real_complex(transpose(Gamma), M5)))
    Y[i3] .= Y[i3] .+ M5

    return Y
end

function precond_3_3_vector(F,invZ,invP,A,Gamma,w,X1,X2,X3)

    n1=length(X1)
    n2=length(X2)
    n3=length(X3)

    i1=range(1, stop=n1)
    i2=range(n1+1,stop=n1+n2)
    i3=range(n1+n2+1,stop=n1+n2+n3)

    Y=zeros(ComplexF64 , n1+n2+n3)

    M1 = prod_real_complex(invZ, X1)
    M2 = F\(prod_real_complex(transpose(A), M1))  
    M3 = prod_real_complex(invP, X2)   
    M4 = F\prod_real_complex(Gamma, M3) 
    M5 = F\X3
    
    Y[i1] .= Y[i1] .+ M1-1.0*(prod_real_complex((invZ),prod_real_complex((A), M2)))
    Y[i1] .= Y[i1] .+ 1im*w*(prod_real_complex((invZ),prod_real_complex((A), M4)))
    Y[i1] .= Y[i1] .- 1.0*(prod_real_complex((invZ),prod_real_complex((A), M5)))
    
    Y[i2] .= Y[i2] .+ (prod_real_complex(invP,prod_real_complex((transpose(Gamma)), M2)))
    Y[i2] .= Y[i2] .+ M3 - 1im*w*(prod_real_complex(invP,prod_real_complex(transpose(Gamma), M4)))
    Y[i2] .= Y[i2] .+ (prod_real_complex(invP,prod_real_complex(transpose(Gamma), M5)))
    
    Y[i3] .= Y[i3] .+ M2
    Y[i3] .= Y[i3] .- 1im*w*M4
    Y[i3] .= Y[i3] .+ M5
    
    Y=convert(Array{Complex{Float64}}, Y)
    return Y
end

function prod_real_complex(A,x)
    # A is a N x N real matrix and x is a complex matrix
    N=size(A,1);
    y=zeros(ComplexF64 , N, 1)
    y=*(A,real.(x))+1im * *(A,imag.(x))
    return y
end

function prod_complex_real(A,x)
    # A is a N x N complex matrix and x is a real matrix
    N=size(A,1);
    y=zeros(ComplexF64 , N, 1)
    y=*(real.(A),x)+1im * *(imag.(A),x)
    return y
end

function s2z(S,Zo)
    num_ports=size(S)[1]
    nfreq=size(S)[3]
    Z = zeros(ComplexF64 , num_ports, num_ports, nfreq)
    Id = Matrix{Int64}(I, num_ports, num_ports)
    for cont in range(1, stop=nfreq)
        Z[:,:,cont]=Zo*((Id-1.0*S[:,:,cont])\(Id+S[:,:,cont]))
    end
    return Z
end

function s2y(S,Zo)
    num_ports=size(S)[1]
    nfreq=size(S)[3]
    Y = zeros(ComplexF64 , num_ports, num_ports, nfreq)
    Id = Matrix{Int64}(I, num_ports, num_ports)
    for cont in range(1, stop=nfreq)
        Y[:,:,cont]=Zo*((Id+S[:,:,cont])\(Id-1.0*S[:,:,cont]))
    end
    return Y
end



s2y (generic function with 1 method)

In [2]:
include("create_volumes_mapping_v2.jl")
include("create_volume_centers.jl")
include("create_Grids_externals.jl")
include("compute_FFT_mutual_coupling_mats.jl")
include("mesher_FFT.jl")

using JSON, DelimitedFiles, JSON3, MAT, JLD2
using MLUtils: unsqueeze


function encode_complex(z)
    if z isa Complex
        return (z.re, z.im)
    else
        error("Object of type $typeof(z) is not JSON serializable")
    end
end
       
function dump_json_data(matrix_Z,matrix_S,matrix_Y, num_ports)

    z = [[[[0.1, 0.0]]]]
    pop!(z)

    matrix_Z = convert(Array{ComplexF64, 3}, matrix_Z)
    matrix_S = convert(Array{ComplexF64, 3}, matrix_S)
    matrix_Y = convert(Array{ComplexF64, 3}, matrix_Y)

    for i in range(1,num_ports)
        for j in range(1, num_ports)
            elements = map(v -> reinterpret(Float64, [v]), matrix_Z[i,j,:])
            push!(z, [elements])
        end
    end

    s = [[[[0.1, 0.0]]]]
    pop!(s)
    
    for i in range(1,num_ports)
        for j in range(1, num_ports)
            elements = map(v -> reinterpret(Float64, [v]), matrix_S[i,j,:])
            push!(s, [elements])
        end
    end

    y = [[[[0.1, 0.0]]]]
    pop!(y)
    
    for i in range(1,num_ports)
        for j in range(1, num_ports)
            elements = map(v -> reinterpret(Float64, [v]), matrix_Y[i,j,:])
            push!(y, [elements])
        end
    end

    solver_matrices_dict = Dict(
        "matrix_Z" => JSON.json(z),
        "matrix_S" => JSON.json(s),
        "matrix_Y" => JSON.json(y)
    )
    
    return solver_matrices_dict
end


Base.@kwdef struct signal
    dict_element
    value = complex(float(dict_element["Re"]),float(dict_element)["Im"])
end
        
Base.@kwdef struct geom_attributes
    dict_element
    radius = dict_element["radius"]
    segments = dict_element["segments"]
end
        
Base.@kwdef struct transf_params
    dict_element
    position = dict_element["position"]
    rotation = dict_element["rotation"]  
    scale = dict_element["scale"]   
end

Base.@kwdef struct element
    dict_element
    name = dict_element["name"]
    type = dict_element["type"]
    keyComponent = dict_element["keyComponent"]
    geometryAttributes = geom_attributes(dict_element["geometryAttributes"])
    transformationParams = transf_params(dict_element["transformationParams"])
end
        
            
        
Base.@kwdef struct port
    dict_element
    name = dict_element["name"]
    type = dict_element["type"]
    inputElement = element(dict_element["inputElement"])
    outputElement = element(dict_element["outputElement"])
    rlcParams = dict_element["rlcParams"]
    isSelected = dict_element["isSelected"]
end

        
Base.@kwdef struct lumped_element
    dict_element
    name = dict_element["name"]
    type = dict_element["type"]
    value = dict_element["value"]
    inputElement = element(dict_element["inputElement"])
    outputElement = element(dict_element["outputElement"])
    rlcParams = dict_element["rlcParams"]
    isSelected = dict_element["isSelected"]
end

Base.@kwdef mutable struct material
    dict_element
    name = dict_element["name"]
    color = dict_element["color"]
    permeability = dict_element["permeability"]
    tangent_delta_permeability = dict_element["tangent_delta_permeability"]
    custom_permeability = dict_element["custom_permeability"]
    permittivity = dict_element["permittivity"]
    tangent_delta_permittivity = dict_element["tangent_delta_permittivity"]
    custom_permittivity = dict_element["custom_permittivity"]
    conductivity = dict_element["conductivity"]
    tangent_delta_conductivity = dict_element["tangent_delta_conductivity"]
    custom_conductivity = dict_element["custom_conductivity"]
    sigmar = dict_element["conductivity"]
    tan_D = dict_element["tangent_delta_permittivity"]
    eps_re = dict_element["permittivity"]
    mur = dict_element["permeability"]
    epsr = 1
    Rx = nothing
    Ry = nothing
    Rz = nothing
    Cx = nothing
    Cy = nothing
    Cz = nothing
end
        
mutable struct port_def
    port_start
    port_end
    port_voxels
    port_nodes
    surf_s_port_nodes
    surf_e_port_nodes
end

mutable struct le_def
    value
    type
    le_start
    le_end
    le_voxels
    le_nodes
    surf_s_le_nodes
    surf_e_le_nodes
end
                

function read_ports(inputData, escal)
    #@assert inputData isa Dict
    ports = inputData["ports"]
    port_objects = [el for el in ports]
    input_positions = []
    output_positions = []
    N_PORTS = length(port_objects)
    
    for port_object in port_objects
        @assert length(port_object.inputElement.transformationParams.position)==3
        ipos = zeros((1,3))
        ipos[1, 1] = port_object.inputElement.transformationParams.position[1]*escal
        ipos[1, 2] = port_object.inputElement.transformationParams.position[2]*escal
        ipos[1, 3] = port_object.inputElement.transformationParams.position[3]*escal
        push!(input_positions, ipos)
        @assert length(port_object.outputElement.transformationParams.position)==3
        opos = zeros((1, 3))
        opos[1, 1] = port_object.outputElement.transformationParams.position[1]*escal
        opos[1, 2] = port_object.outputElement.transformationParams.position[2]*escal
        opos[1, 3] = port_object.outputElement.transformationParams.position[3]*escal
        push!(output_positions, opos)
    end
    @assert length(input_positions)==N_PORTS && length(output_positions)==N_PORTS
    inp_pos = []
    for i in input_positions
        push!(inp_pos, unsqueeze([i], dims=2))
    end
    out_pos = []
    for i in output_positions
        push!(out_pos, unsqueeze([i], dims=2))
    end
    ports_out = port_def(inp_pos, out_pos,zeros(Int64, (N_PORTS, 2)),zeros(Int64,(N_PORTS, 2)), Array{Any}(undef,0), Array{Any}(undef,0))
    
    return ports_out
end


function read_lumped_elements(inputData, escal)
    
    #@assert inputData isa Dict
    
    lumped_elements = inputData["lumped_elements"]
    
    lumped_elements_objects = [el for el in lumped_elements]
    input_positions = []
    output_positions = []
    values = []
    types = []
    N_LUMPED_ELEMENTS = length(lumped_elements_objects)
    if N_LUMPED_ELEMENTS == 0
        lumped_elements_out = le_def(zeros(0),zeros(Int64, 0),zeros((0, 3)),zeros((0, 3)),zeros(Int64, (0, 2)),zeros(Int64, (0, 2)), Array{Any}(undef,0), Array{Any}(undef,0))
        @assert length(input_positions)==N_LUMPED_ELEMENTS && length(output_positions)==N_LUMPED_ELEMENTS && length(values)==N_LUMPED_ELEMENTS && length(types)==N_LUMPED_ELEMENTS
    else
        for lumped_element_object in lumped_elements_objects
            @assert length(lumped_element_object.inputElement.transformationParams.position)==3
            ipos = zeros((1,3))
            ipos[1, 1] = lumped_element_object.inputElement.transformationParams.position[1]*escal
            ipos[1, 2] = lumped_element_object.inputElement.transformationParams.position[2]*escal
            ipos[1, 3] = lumped_element_object.inputElement.transformationParams.position[3]*escal
            push!(input_positions, ipos)
            @assert length(lumped_element_object.outputElement.transformationParams.position)==3
            opos = zeros((1, 3))
            opos[1, 1] = lumped_element_object.outputElement.transformationParams.position[1]*escal
            opos[1, 2] = lumped_element_object.outputElement.transformationParams.position[2]*escal
            opos[1, 3] = lumped_element_object.outputElement.transformationParams.position[3]*escal
            push!(output_positions, opos)            
            lvalue = zeros(1)
            lvalue[1] = lumped_element_object.value
            append!(values, lvalue)
            
            ltype = zeros(Int64, 1)
            ltype[1] = lumped_element_object.type
            push!(types, ltype)
        end
    end
            
        @assert length(input_positions)==N_LUMPED_ELEMENTS && length(output_positions)==N_LUMPED_ELEMENTS && length(values)==N_LUMPED_ELEMENTS && length(types)==N_LUMPED_ELEMENTS
    
        value = []
        for i in values
            push!(value, i[1])
        end

        type = []
        for i in types
            push!(type, i[1])
        end

        in_pos = []
        for i in input_positions
            push!(in_pos, unsqueeze([i], dims=2))
        end

        out_pos = []
        for i in output_positions
            push!(out_pos, unsqueeze([i], dims=2))
        end

        lumped_elements_out = le_def(value,type,in_pos, out_pos,zeros(Int64, (N_LUMPED_ELEMENTS, 2)), (Int64, (N_LUMPED_ELEMENTS, 2)), Array{Any}(undef,0), Array{Any}(undef,0))

    return lumped_elements_out
end

function read_materials(inputData)
    #@assert inputData isa Dict
    materials = inputData["materials"]
    # for el in materials
    #     el["sigmar"] = 9.4e6
    #     el["tan_D"] = 0
    #     el["eps_re"] = 1
    #     el["mur"] = 1
    # end
    materials_objects = [material(dict_element=el) for el in materials]
    return materials_objects
end

function read_signals(inputData)
    #@assert inputData isa Dict
    signals = inputData["signals"]
    signals_objects = [el for el in signals]
    return signals_objects
end

struct GMRES_set
    Inner_Iter
    Outer_Iter
    tol
end

function getEscalFrom(unit)
    escal = 1
    if (unit == "m")
        escal = 1
    end
    if (unit == "dm")
        escal = 1e-1
    end
    if (unit == "cm")
        escal = 1e-2
    end
    if (unit == "mm")
        escal = 1e-3
    end
    if (unit == "microm")
        escal = 1e-6
    end
    if (unit == "nanom")
        escal = 1e-9
    end
    return escal
end

    
function doSolving(mesherDict, inputDict, unit)    

    escal = getEscalFrom(unit)

    #@save "/Users/edgardovittoria/Downloads/spiraleInput3.jl" mesherDict inputDict unit

    

    
    sx, sy, sz = mesherDict["cell_size"]["cell_size_x"]*1000*escal,mesherDict["cell_size"]["cell_size_y"]*1000*escal,mesherDict["cell_size"]["cell_size_z"]*1000*escal
   

    origin_x = mesherDict["origin"]["origin_x"]
    origin_y = mesherDict["origin"]["origin_y"]
    origin_z = mesherDict["origin"]["origin_z"]

    origin = (origin_x,origin_y,origin_z)
    Nx = Int64(mesherDict["n_cells"]["n_cells_x"])
    Ny = Int64(mesherDict["n_cells"]["n_cells_y"])
    Nz = Int64(mesherDict["n_cells"]["n_cells_z"])

    testarray = []
    for (index, value) in mesherDict["mesher_matrices"]
        push!(testarray, copy(value))
    end


    grids = []

    for values in testarray
        if (length(testarray) == 1)
            grids = unsqueeze([values], dims=2)
        else
            push!(grids, unsqueeze(values, dims=2))
        end
    end


    frequencies = inputDict["frequencies"]
    freq = Array{Float64}(undef, 1, length(frequencies))
    for i in range(1, length(frequencies))
        freq[1,i] = frequencies[i]
    end
    #freq = convert(Array{Float64}, freq)

    n_freq = length(freq)
    
    PORTS = read_ports(inputDict, escal)

    L_ELEMENTS = read_lumped_elements(inputDict, escal)

    MATERIALS = read_materials(inputDict) 
    SIGNALS = read_signals(inputDict)
    
    
    
    
    # # START SETTINGS--------------------------------------------
    
    inner_Iter = 100
    outer_Iter = 1
    tol = 1e-4*ones((n_freq))
    # ind_low_freq= filter(i -> !iszero(freq[i]), findall(f -> f<1e5, freq))
    # tol[ind_low_freq] .= 1e-7
    

    GMRES_settings = GMRES_set(inner_Iter,outer_Iter,tol)
    QS_Rcc_FW=1; # 1 QS, 2 Rcc, 3 Taylor
    use_escalings=1;

    
   
    #display(jsonmesher)
    mapping_vols,num_centri=create_volumes_mapping_v2(grids)
    
    #centri_vox,id_mat=create_volume_centers(grids,mapping_vols,num_full_vox,sx,sy,sz);
    centri_vox,id_mat=create_volume_centers(grids,mapping_vols,num_centri,sx,sy,sz,origin);


    externals_grids=create_Grids_externals(grids);
    escalings,incidence_selection,circulant_centers,diagonals,expansions,ports,lumped_elements,li_mats,Zs_info=mesher_FFT(use_escalings,MATERIALS,sx,sy,sz,grids,centri_vox,externals_grids,mapping_vols,PORTS,L_ELEMENTS, origin);
    
    # matwrite("/Users/edgardovittoria/Downloads/matfile.mat", Dict(
    #     "grids" =>  grids,
    #     "num_full_vox" => num_centri,
    #     "sx" => sx,
    #     "sy" => sy,
    #     "sz" => sz
    # ))
    FFTCP,FFTCLp= @time compute_FFT_mutual_coupling_mats(circulant_centers,escalings,Nx,Ny,Nz,QS_Rcc_FW);
    #println(FFTCLp)
    
    println("time for solver")
    out = @time FFT_solver_QS_S_type(freq,escalings,incidence_selection,FFTCP,FFTCLp,diagonals,ports,lumped_elements,expansions,GMRES_settings,Zs_info,QS_Rcc_FW);
    #println(out)
    return dump_json_data(out["Z"],out["S"],out["Y"], length(inputDict["ports"]))
end



doSolving (generic function with 1 method)

In [11]:
using JLD2
@load "/Users/edgardovittoria/Downloads/crossbarInput.jl" mesherDict inputDict unit
println("Running Test crossbar 1")
doSolving(mesherDict, inputDict, unit)


Running Test crossbar 1
  0.868922 seconds (29.95 M allocations: 684.263 MiB, 6.39% gc time)
time for solver
6
Freq n=1 - Freq Tot=10
Converged after 8 iterations.
Converged after 8 iterations.
Freq n=2 - Freq Tot=10
Converged after 14 iterations.
Converged after 14 iterations.
Freq n=3 - Freq Tot=10
Converged after 7 iterations.
Converged after 7 iterations.
Freq n=4 - Freq Tot=10
Converged after 7 iterations.
Converged after 7 iterations.
Freq n=5 - Freq Tot=10
Converged after 9 iterations.
Converged after 9 iterations.
Freq n=6 - Freq Tot=10
Converged after 11 iterations.
Converged after 11 iterations.
Freq n=7 - Freq Tot=10
Converged after 13 iterations.
Converged after 13 iterations.
Freq n=8 - Freq Tot=10
Converged after 16 iterations.
Converged after 16 iterations.
Freq n=9 - Freq Tot=10
Converged after 19 iterations.
Converged after 19 iterations.
Freq n=10 - Freq Tot=10
Converged after 23 iterations.
Converged after 24 iterations.
 14.992277 seconds (1.48 M allocations: 19.633

Dict{String, String} with 3 entries:
  "matrix_Z" => "[[[[25.0966484770434,0.0009496074287147255],[25.09664443777136…
  "matrix_S" => "[[[[-0.49614907092313587,3.994040273127039e-5],[-0.49614910084…
  "matrix_Y" => "[[[[6491.28569757601,-68.60189126965025],[6488.615744285878,-1…

  1	 10	3.03e-03
  1	 11	1.95e-03
  1	 12	7.78e-04
  1	 13	4.90e-04


  1	 14	3.05e-04

Converged after 14 iterations.
=== gmres ===
rest	iter	resnorm


  1	  1	3.81e+00
  1	  2	3.74e+00


...

  1	 16	7.50e-01
  1	 17	5.35e-01
  1	 18	2.80e-01
  1	 19	2.06e-01


  1	 20	1.46e-01
  1	 21	1.06e-01


  1	 22	8.10e-02
  1	 23	5.67e-02


  1	 24	3.26e-02

Converged after 24 iterations.
 14.178175 seconds (336.39 k allocations: 19.562 GiB, 7.03% gc time)


Dict{String, String} with 3 entries:
  "matrix_Z" => "[[[[25.0966484770434,0.0009496074287147255],[25.09664443777136…
  "matrix_S" => "[[[[-0.49614907092313587,3.994040273127039e-5],[-0.49614910084…
  "matrix_Y" => "[[[[6491.28569757601,-68.60189126965025],[6488.615744285878,-1…

In [9]:
using JLD2
@load "/Users/edgardovittoria/Downloads/crossbarInput2.jl" mesherDict inputDict unit
println("Running Test crossbar 2")
doSolving(mesherDict, inputDict, unit)

Running Test crossbar 2
  5.616977 seconds (194.94 M allocations: 4.450 GiB, 6.42% gc time)
time for solver
6
Freq n=1 - Freq Tot=10
Converged after 9 iterations.
Converged after 9 iterations.
Freq n=2 - Freq Tot=10
Converged after 22 iterations.
Converged after 19 iterations.
Freq n=3 - Freq Tot=10
Converged after 8 iterations.
Converged after 8 iterations.
Freq n=4 - Freq Tot=10
Converged after 9 iterations.
Converged after 10 iterations.
Freq n=5 - Freq Tot=10
Converged after 12 iterations.
Converged after 11 iterations.
Freq n=6 - Freq Tot=10
Converged after 14 iterations.
Converged after 14 iterations.
Freq n=7 - Freq Tot=10
Converged after 16 iterations.
Converged after 16 iterations.
Freq n=8 - Freq Tot=10
Converged after 22 iterations.
Converged after 22 iterations.
Freq n=9 - Freq Tot=10
Converged after 26 iterations.
Converged after 26 iterations.
Freq n=10 - Freq Tot=10
Converged after 30 iterations.
Converged after 30 iterations.
 72.431848 seconds (4.71 M allocations: 172.

Dict{String, String} with 3 entries:
  "matrix_Z" => "[[[[25.09029618414711,0.0009044870023106877],[25.0902826009744…
  "matrix_S" => "[[[[-0.4964015464160233,3.8052292442619926e-5],[-0.49640167949…
  "matrix_Y" => "[[[[6946.8839040746925,-74.84882334386359],[6943.914028568541,…

In [12]:
using JLD2
@load "/Users/edgardovittoria/Downloads/crossbarInput3.jl" mesherDict inputDict unit
println("Running Test crossbar 3")
doSolving(mesherDict, inputDict, unit)

Running Test crossbar 3
 20.331175 seconds (674.98 M allocations: 20.471 GiB, 7.75% gc time)
time for solver
6
Freq n=1 - Freq Tot=10
Converged after 11 iterations.
Converged after 11 iterations.
Freq n=2 - Freq Tot=10
Converged after 29 iterations.
Converged after 33 iterations.
Freq n=3 - Freq Tot=10
Converged after 11 iterations.
Converged after 12 iterations.
Freq n=4 - Freq Tot=10
Converged after 15 iterations.
Converged after 15 iterations.
Freq n=5 - Freq Tot=10
Converged after 17 iterations.
Converged after 17 iterations.
Freq n=6 - Freq Tot=10
Converged after 21 iterations.
Converged after 21 iterations.
Freq n=7 - Freq Tot=10
Converged after 25 iterations.
Converged after 25 iterations.
Freq n=8 - Freq Tot=10
Converged after 30 iterations.
Converged after 30 iterations.
Freq n=9 - Freq Tot=10
Converged after 37 iterations.
Converged after 38 iterations.
Freq n=10 - Freq Tot=10
Converged after 44 iterations.
Converged after 44 iterations.
894.110674 seconds (1.96 M allocations

Dict{String, String} with 3 entries:
  "matrix_Z" => "[[[[25.092887452786815,0.0007390904828431585],[25.092859952592…
  "matrix_S" => "[[[[-0.4962990303588418,3.614306340881113e-5],[-0.496299341682…
  "matrix_Y" => "[[[[6755.014947586592,-70.14575769178292],[6752.164208219463,-…

In [3]:
using JLD2
@load "/Users/edgardovittoria/Downloads/wptInput.jl" mesherDict inputDict unit
println("Running Test wpt 1")
doSolving(mesherDict, inputDict, unit)

Running Test wpt 1
  3.834642 seconds (32.79 M allocations: 1.374 GiB, 3.73% gc time, 79.43% compilation time)
time for solver
6
Freq n=1 - Freq Tot=10
Converged after 20 iterations.
Freq n=2 - Freq Tot=10
Converged after 20 iterations.
Freq n=3 - Freq Tot=10
Converged after 19 iterations.
Freq n=4 - Freq Tot=10
Converged after 26 iterations.
Freq n=5 - Freq Tot=10
Converged after 21 iterations.
Freq n=6 - Freq Tot=10
Converged after 46 iterations.
Freq n=7 - Freq Tot=10
Converged after 51 iterations.
Freq n=8 - Freq Tot=10
Converged after 38 iterations.
Freq n=9 - Freq Tot=10
Converged after 55 iterations.
Freq n=10 - Freq Tot=10
Converged after 73 iterations.
 60.759400 seconds (24.36 M allocations: 97.366 GiB, 4.86% gc time, 6.88% compilation time)


Dict{String, String} with 3 entries:
  "matrix_Z" => "[[[[25.005681746335245,-0.0005237416422886069],[25.00813705366…
  "matrix_S" => "[[[[-0.333232332096097,-9.309551955354323e-6],[-0.333188689590…
  "matrix_Y" => "[[[[99.97727813478507,0.0020940146472723633],[99.9674619008460…

In [ ]:
using JLD2
@load "/Users/edgardovittoria/Downloads/wptInput2.jl" mesherDict inputDict unit
println("Running Test wpt 2")
doSolving(mesherDict, inputDict, unit)

Running Test wpt 2
  5.543879 seconds (171.46 M allocations: 7.253 GiB, 12.06% gc time)
time for solver
6
Freq n=1 - Freq Tot=10
Converged after 25 iterations.
Freq n=2 - Freq Tot=10


In [ ]:
using JLD2
@load "/Users/edgardovittoria/Downloads/antennaInput.jl" mesherDict inputDict unit
println("Running Test antenna 1")
doSolving(mesherDict, inputDict, unit)

In [ ]:
using JLD2
@load "/Users/edgardovittoria/Downloads/antennaInput2.jl" mesherDict inputDict unit
println("Running Test antenna 2")
doSolving(mesherDict, inputDict, unit)

In [ ]:
using JLD2
@load "/Users/edgardovittoria/Downloads/antennaInput3.jl" mesherDict inputDict unit
println("Running Test antenna 3")
doSolving(mesherDict, inputDict, unit)

In [ ]:
using JLD2
@load "/Users/edgardovittoria/Downloads/copperLoopInput.jl" mesherDict inputDict unit
println("Running Test copper Loop 1")
doSolving(mesherDict, inputDict, unit)

In [ ]:
using JLD2
@load "/Users/edgardovittoria/Downloads/copperLoopInput2.jl" mesherDict inputDict unit
println("Running Test copper Loop 2")
doSolving(mesherDict, inputDict, unit)

In [ ]:
using JLD2
@load "/Users/edgardovittoria/Downloads/copperLoopInput3.jl" mesherDict inputDict unit
println("Running Test copper Loop 3")
doSolving(mesherDict, inputDict, unit)


In [ ]:
using JLD2
@load "/Users/edgardovittoria/Downloads/spiraleInput.jl" mesherDict inputDict unit
println("Running Test spirale 1")
doSolving(mesherDict, inputDict, unit)


In [ ]:
using JLD2
@load "/Users/edgardovittoria/Downloads/spiraleInput2.jl" mesherDict inputDict unit
println("Running Test spirale 2")
doSolving(mesherDict, inputDict, unit)

In [ ]:
using JLD2

@load "/Users/edgardovittoria/Downloads/spiraleInput3.jl" mesherDict inputDict unit
println("Running Test spirale 3")
doSolving(mesherDict, inputDict, unit)